In [ ]:
***************************************

In [1]:
import torch
import torchvision
import cv2
import numpy as np
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
from jetbot import Robot, Camera, bgr8_to_jpeg
import threading
import time

# Nastavení zařízení pro AI
device = torch.device('cuda')
print("Knihovny načteny a CUDA zařízení připraveno.")

Knihovny načteny a CUDA zařízení připraveno.


In [2]:
# 1. Inicializace Robota
robot = Robot()

# 2. Vytvoření ovládacích prvků (UI)
left_slider = widgets.FloatSlider(min=-1.0, max=1.0, description='Levý motor', orientation='vertical')
right_slider = widgets.FloatSlider(min=-1.0, max=1.0, description='Pravý motor', orientation='vertical')

# 3. Propojení (Traitlets)
# Změna slideru -> změna motoru
left_link = traitlets.link((left_slider, 'value'), (robot.left_motor, 'value'))
right_link = traitlets.link((right_slider, 'value'), (robot.right_motor, 'value'))

# Zobrazení pro test
print("Zkuste pohnout posuvníky pro test motorů:")
display(widgets.HBox([left_slider, right_slider]))

Zkuste pohnout posuvníky pro test motorů:


In [4]:
# Index 0 je obvykle první připojený ovladač (Xbox/PlayStation/Logitech)
controller = widgets.Controller(index=0)

print("Níže by se měl objevit widget ovladače.")
print("Pokud nevidíte změny při stisku tlačítek, klikněte na tuto stránku a stiskněte tlačítko na gamepadu.")
display(controller)

Níže by se měl objevit widget ovladače.
Pokud nevidíte změny při stisku tlačítek, klikněte na tuto stránku a stiskněte tlačítko na gamepadu.


Controller()

In [5]:
# 1. Inicializace Kamery
try:
    camera = Camera.instance(width=224, height=224)
    print("Kamera inicializována.")
except Exception as e:
    print(f"Chyba kamery: {e}")

# 2. Widget pro zobrazení obrazu
image_widget = widgets.Image(format='jpeg', width=224, height=244)

# Propojení kamery s widgetem (přímý stream)
camera_link = traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

# 3. Načtení AI Modelu
model_path = 'our_best_model_batch_2_50_50.pth'  # <--- ZDE ZKONTROLUJTE NÁZEV SOUBORU
try:
    model = torchvision.models.resnet18(pretrained=False)
    model.fc = torch.nn.Linear(512, 2)
    model.load_state_dict(torch.load(model_path))
    model = model.to(device)
    model.eval()
    print("AI Model úspěšně načten.")
except FileNotFoundError:
    print(f"CHYBA: Soubor {model_path} nebyl nalezen!")

# Preprocessing konstanty
mean = torch.Tensor([0.485, 0.456, 0.406]).cuda()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda()

def preprocess(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = torch.from_numpy(image).float().permute(2, 0, 1) / 255.0
    image = image.to(device)
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

display(image_widget)

Kamera inicializována.
AI Model úspěšně načten.


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [6]:
import time
from collections import deque # Knihovna pro efektivní historii hodnot

# --- Globální proměnné pro řízení ---
speed_scale = 0.17
is_avoiding = False  
running = True

# --- NASTAVENÍ KLOUZAVÉHO PRŮMĚRU ---
history_len = 5  # Kolik snímků zpětně průměrovat (čím víc, tím plynulejší, ale pomalejší reakce)
# Vytvoříme frontu naplněnou nulami
stop_history = deque([0.0] * history_len, maxlen=history_len)

# UI prvky
prob_label = widgets.Label(value="Čekám na data...")
# Přidáme druhý slider pro raw data, abyste viděl rozdíl
raw_slider = widgets.FloatSlider(description='Okamžitá', min=0.0, max=1.0, orientation='vertical', readout_format='.2f')
avg_slider = widgets.FloatSlider(description='PRŮMĚR', min=0.0, max=1.0, orientation='vertical', readout_format='.2f')

# --- Funkce pro zpracování obrazu (AI MOZEK) ---
def vision_loop():
    global is_avoiding, running, stop_history
    
    while running:
        try:
            # 1. Získání obrazu a predikce
            image = camera.value
            if image is not None:
                preprocessed = preprocess(image)
                with torch.no_grad():
                    output = model(preprocessed)
                    prediction = torch.nn.functional.softmax(output, dim=1)
                
                # Získání okamžité hodnoty (Class 1 = Stop)
                raw_stop_conf = prediction[0, 1].item()
                
                # --- VÝPOČET KLOUZAVÉHO PRŮMĚRU ---
                stop_history.append(raw_stop_conf) # Přidá novou, starou automaticky zahodí
                avg_stop_conf = sum(stop_history) / len(stop_history) # Spočítá průměr
                
                # Aktualizace UI (ukazujeme obě hodnoty)
                raw_slider.value = raw_stop_conf
                avg_slider.value = avg_stop_conf
                
                # 2. Rozhodovací logika (Nyní se rozhodujeme podle PRŮMĚRU)
                if avg_stop_conf > 0.80 and not is_avoiding:
                    is_avoiding = True 
                    
                    prob_label.value = f"⚠️ PŘEKÁŽKA (Avg: {avg_stop_conf:.2f})"
                    
                    # --- AUTOMATICKÝ MANÉVR ---
                    left_slider.value = -0.3
                    right_slider.value = 0.3
                    
                    time.sleep(0.5) 
                    
                    left_slider.value = 0
                    right_slider.value = 0
                    prob_label.value = f"🛑 ZASTAVENO. Převezměte řízení."
                    
                    # Vyčistíme historii, aby se robot hned nerozjel, pokud tam překážka stále je,
                    # nebo naopak aby hned nezastavil po vrácení kontroly.
                    # Naplníme ji nulami (nebo aktuální hodnotou), aby průměr začal nanovo.
                    stop_history.extend([0.0] * history_len)
                    
                    time.sleep(1.0) 
                    
                    is_avoiding = False 
                    
                elif not is_avoiding:
                    prob_label.value = f"✓ JEDU (Avg: {avg_stop_conf:.2f})"
            
            time.sleep(0.05)
            
        except Exception as e:
            pass

# --- Funkce pro ovládání motorů (GAMEPAD) ---
def update_motors(change):
    global is_avoiding
    
    if is_avoiding:
        return

    x = controller.axes[0].value
    y = -controller.axes[1].value 

    if abs(x) < 0.1: x = 0
    if abs(y) < 0.1: y = 0

    left = y + x
    right = y - x

    max_val = max(abs(left), abs(right))
    if max_val > 1.0:
        left /= max_val
        right /= max_val

    left_slider.value = left * speed_scale
    right_slider.value = right * speed_scale

# --- SPUŠTĚNÍ ---
controller.axes[0].unobserve_all()
controller.axes[1].unobserve_all()

controller.axes[0].observe(update_motors, names='value')
controller.axes[1].observe(update_motors, names='value')

vision_thread = threading.Thread(target=vision_loop)
vision_thread.daemon = True
vision_thread.start()

print("Systém spuštěn s KLOUZAVÝM PRŮMĚREM (Stabilizace detekce).")
display(widgets.HBox([
    widgets.VBox([prob_label, raw_slider]), # Ukazuje okamžitou hodnotu
    widgets.VBox([widgets.Label("Průměr"), avg_slider]), # Ukazuje vyhlazenou hodnotu (rozhodovací)
    widgets.VBox([widgets.Label("Motory"), widgets.HBox([left_slider, right_slider])])
]))

Systém spuštěn s KLOUZAVÝM PRŮMĚREM (Stabilizace detekce).


In [7]:
# Zastavení smyček
running = False
time.sleep(0.5) # Počkáme chvíli na doběhnutí vlákna

# Odpojení posluchačů (aby ovladač přestal posílat příkazy)
try:
    controller.axes[0].unobserve_all()
    controller.axes[1].unobserve_all()
except:
    pass

# Odpojení traitlets
try:
    left_link.unlink()
    right_link.unlink()
    camera_link.unlink()
except:
    pass

# Zastavení hardwaru
robot.stop()
camera.stop()

print("Robot bezpečně zastaven a odpojen.")

Robot bezpečně zastaven a odpojen.
